Data cleaning to relational tables

In [ ]:
sql = """create table minister
(
    id                integer primary key,
    "wiki link"       TEXT,
    start             TEXT,
    district_link     TEXT,
    "group"           TEXT,
    member            TEXT,
    party             TEXT,
    district          TEXT,
    is_senator        INT,
    is_representative INT,
    graduated INT
);

insert into minister("wiki link", start, district_link, "group", member, party, district, is_senator,
                            is_representative, graduated)
select distinct "wiki link",
       start,
       district_link,
       "group",
       member,
       party,
       district,
       is_senator,
       is_representative,
       graduated,
       mp_id
from cleaning_data_ministers;

ALTER TABLE minister ADD COLUMN mp_id integer;
UPDATE minister
SET mp_id = a.mp_id from (SELECT distinct member, mp_id from ministers) a WHERE a.member = minister.member;


create TABLE minister as SELECT distinct "wiki link", start, district_link, "group", member, party, district, is_senator, is_representative, graduated from ministers;
ALTER TABLE minister ADD COLUMN mp_id integer;
UPDATE minister
SET mp_id = a.mp_id from (SELECT distinct member, mp_id from ministers) a WHERE a.member = minister.member;

create TABLE minister_education as SELECT distinct ministers.education_id, minister.id as minister_id  from ministers join minister on ministers.member = minister.member;

CREATE TABLE education_acara as SELECT e.fid as education_id,  "ACARA SML ID" as acara_id from acara__education JOIN education e on acara__education.school_name = e.school_name;

drop table  if exists education_acara_dg_tmp;
create table education_acara_dg_tmp
(
    education_id INT not null
        constraint education_acara_education_fid_fk
            references education,
    acara_id     integer not null
        constraint education_acara_acara_fid_fk
            references acara_school_locations_2022,
    id           integer primary key /*autoincrement needs PK*/
);

insert into education_acara_dg_tmp(education_id, acara_id, id)
select education_id, asl.fid as acara_id, id
from education_acara JOIN acara_school_locations_2022 asl on asl."ACARA SML ID" = education_acara.acara_id;

drop table education_acara;

alter table education_acara_dg_tmp
    rename to education_acara;

alter table ministers rename to  cleaning_data_ministers;

drop table  if exists education_acara_dg_tmp;
create table education_acara_dg_tmp
(
    education_id INT not null
        constraint education_acara_education_fid_fk
            references education,
    acara_id     integer not null
        constraint education_acara_acara_fid_fk
            references acara_school_locations_2022,
    id           integer primary key /*autoincrement needs PK*/
);

insert into education_acara_dg_tmp(education_id, acara_id)
select e.fid as education_id, asl.fid as acara_id
from acara__education ae
    JOIN acara_school_locations_2022 asl on asl."ACARA SML ID" = ae."ACARA SML ID"
    JOIN education e  on e.school_name = ae.school_name;
;

drop table education_acara;

alter table education_acara_dg_tmp
    rename to education_acara;

"""


Add some nice views for analysis



In [ ]:
# add mary doyle manually
INSERT INTO minister_education(minister_id, education_id)
SELECT id,fid  from minister,  education
where member = 'Mary Doyle' AND  school_name IN  ('Bendigo Senior Secondary College', 'St Joseph’s College,  Echuca' );

INSERT INTO education_acara(education_id, acara_id)
SELECT e.fid, a.fid from education e , acara_school_locations_2022 a
where school_name IN  ('Bendigo Senior Secondary College', 'St Joseph’s College, Echuca' )
AND ( "School Name" LIKE '%joseph%' AND "Suburb" = 'ECHUCA' OR "School Name" = 'Bendigo Senior Secondary College')